# Load and initialize

settings

In [ ]:
# sample number
j=str(12345)
n_samples = 1772
    # create visualization
visualize = True
    # show figures
show = True

load_from_file = False

envirement

In [ ]:
import os
import time 
start = time.process_time()
import pathlib
import sys
from IPython.display import Image, display
from tqdm import tqdm_notebook,tqdm
# import time

import numpy as np
# import matplotlib
import matplotlib.pyplot as plt
from matplotlib.pyplot import rc_context

plt.rcParams["figure.figsize"] = (5.0, 4.0)  # Set default size of plots.
plt.rcParams["image.interpolation"] = "nearest"
plt.rcParams["image.cmap"] = "gray"
%matplotlib inline

import pandas as pd
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 180) #according to screen width
# from pandas.core.base import PandasObject
# PandasObject.view = view #allows view meth pd.view

# from scipy import signal, stats
# import seaborn as sns

# from sklearn import metrics

# from sklearn.preprocessing import StandardScaler, KBinsDiscretizer
# from sklearn.metrics import silhouette_samples,silhouette_score
# from sklearn.neighbors import kneighbors_graph

# from lmfit import minimize, Parameters


parent_dir = os.path.abspath(os.path.join(os.getcwd(), os.pardir))
sys.path.append(parent_dir+'/functions/')


from usefull_functions import *
from functions import *
from plot_functions import *
from clustering_functions import *



%load_ext autoreload
%autoreload 2
%reload_ext autoreload
%autoreload 2


dir_data = parent_dir+'/Data/'
# 'Data/'
dir_plots = parent_dir+'/Plots_sample'+j+'/'
saveSVG  =False
settings =        (dir_plots,show,saveSVG)
folderExists(dir_plots)

TF_ENABLE_ONEDNN_OPTS=0

load data; normalized (CD45-)  

In [ ]:
def name_remove(NamesAll):
    for h in ['NCad','ECad','panKeratin',]:
        try:
            NamesAll.remove(h)
        except:
            pass
    return NamesAll
dict = pickle_load('k4_names_dict',dir_data )
NamesAll = name_remove(dict['NamesAll'])
CellIden = name_remove(dict['CellIden'])
EpiCols = name_remove(dict['EpiCols'])
Core = name_remove(dict['Core'])

if load_from_file:
    k_orig = pickle_load('k'+j+'_sample',dir_data )['k']
else:
# from functools import reduce
    k_orig= pd.DataFrame(columns =NamesAll)
#     n_samples = 10000 

    for i in ['4','5','3','2','1']:
    # for j in ['2','2']:  
        k_i =pickle_load('k'+i+'_dict',dir_data )['k'][NamesAll]
        print(i,' :',len(k_i))
        # downsample
        k_i= subsample_k(k_i.copy(),n=n_samples)
        k_i ['sample_num'] = int(i)
        k_orig = k_orig.append(k_i, ignore_index=True)
    
# print(len(k_orig)  )  

print (len(k_orig))    
# print(df)
# j=str(245)
dict ={}
dict['k'] = k_orig

pickle_dump('k'+j+'_sample', dict,dir_data)

CellIden_EpiCols = CellIden + EpiCols
groups = [['CellIden','EpiCols','CellIden+EpiCols'],[CellIden,EpiCols,CellIden_EpiCols]]
del dict

Visualize data

no point to print data.describe() as it is whitened (std~1, mean~0) 

In [ ]:
if show:

    for name in [NamesAll,Core,CellIden,EpiCols]:
        print(len(name))
        print(name)


# Cell Identity

In [ ]:
k = k_orig.copy()
CAll=pd.concat([k]).copy()
s ={}
s['min_dist'] = {}
s['n_neighbors'] = {}
s['min_samples'] = {}
s['eps'] = {}


name = 'Tumor'+j+'_CellIdentity_'


### UMAP

plot for each feature according to its intensity

In [ ]:
s['min_dist']['245']=0.001
s['n_neighbors']['245']=10
s['min_dist']['1245']=0.001
s['n_neighbors']['1245']=10

min_dist = s['min_dist'][j]
n_neighbors = s['n_neighbors'][j]



umapData = calculate_umap(CAll[CellIden],n_neighbors, min_dist)
if visualize:
  

  drawUMAP(umapData, ['H4'],CAll,settings,
              title = f'min_dist = {min_dist}, n_neighbors = {n_neighbors}',
              Figname = name+'1_UMAP_(core)')

# umap_params(k,CellIden,dir_plots,
#             valA = min_dist, valB = n_neighbors
#             )

if j=='4':
    ind  = np.where(umapData[:,0]<5)
    k_orig.drop(k_orig.index[ind],inplace=True)
    k = k_orig.copy();CAll=pd.concat([k]).copy()
    
    umapData = calculate_umap(CAll[CellIden],n_neighbors, min_dist)
    drawUMAP(umapData, ['H4'],CAll,settings,
                title = f'min_dist = {min_dist}, n_neighbors = {n_neighbors}',
                Figname = name+'1_UMAP_(core)')
    


In [ ]:
drawUMAP_intVals(umapData, 'sample_num',CAll,settings,
        title ='Tumor '+j+' Epigen +Cell identity UMAP', 
        Figname = name+'1_UMAP_(samples)')


### Dbscan and clusters

In [ ]:
s['min_samples']['245']=50
s['eps']['245']=0.19
s['min_samples']['1245']=40
s['eps']['1245']=0.19

eps=s['eps'][j];min_samples=s['min_samples'][j]
dbscanData,labels,core_samples_mask = calculate_dbscan(umapData,eps,min_samples)
if visualize:
  drawDbscan(dbscanData,labels,core_samples_mask,settings,
              title=f'min_samples = {min_samples}, eps = {eps}',
              figname = name+'1_dbscan')
              
  


# dbscan_params(X_2d,
#                   valA = eps, valB = min_samples,
#                   itersA = 3, itersB = 3,
#                   )


In [ ]:

plotClusters(k,umapData,labels,NamesAll,settings,
             title = 'Clusters Tumor '+j+' - Cell Iden Based',
             figname = name+'1_Clusters')
             



### HeatMap

In [ ]:
if j=='2':
            K = k.copy() 
            k = pd.concat([k.copy(),kCD45pos2])
            
if visualize:
    for group_name,group in zip(groups[0],groups[1]):
        group_name,group = groups[0][-1],groups[1][-1]
        HeatMap(k[k.Clust!=-1],group,settings,
                title = 'T'+j+' Cell Iden Based: '+group_name,
                figname = name+'1_HeatMap_'+group_name)
if j=='2':
    k=K
    K = None
    # del K

### MeanDist

In [ ]:

if visualize:
    for group_name,group in zip(groups[0],groups[1]):
        MeanDist(k [k.Clust==0],k [k.Clust==1],group,settings,
                title='MeanDist CellIdentity based: '+group_name,
                figname = name+'1_MeanDist_'+group_name)


## features on UMAP

In [ ]:
vals = np.unique(CAll.Clust); vals = vals[vals!=-1]
for val in vals:
    ind = CAll.Clust == val
    drawUMAP_intVals(umapData[ind], 'sample_num',CAll[ind],settings,
        title ='Tumor '+j+' Epigen +Cell identity UMAP' + val, 
        Figname = name+'1_UMAP'+val|'_(samples)')
if visualize:
    drawUMAP(umapData, NamesAll,CAll,settings,
            title ='Tumor'+j+ ' Cell Iden UMAP', 
            Figname = name+'2_UMAP_')

## save data to file

In [ ]:

dbscanData, umapData,k, CAll,labels,s = None,None,None,None,None,None


# Epigenetics Based

In [ ]:


k = k_orig.copy()
CAll=pd.concat([k]).copy()

s ={}
s['min_dist'] = {}
s['n_neighbors'] = {}
s['min_samples'] = {}
s['eps'] = {}

name = 'Tumor'+j+'_Epigen_'



### UMAP

plot for each feature according to its intensity

In [ ]:
s['min_dist']['245']=0.001
s['n_neighbors']['245']=10
s['min_dist']['1245']=0.001
s['n_neighbors']['1245']=10

min_dist = s['min_dist'][j]
n_neighbors = s['n_neighbors'][j]


umapData = calculate_umap(CAll[EpiCols],n_neighbors, min_dist)
if visualize:
    drawUMAP(umapData, ['H4'],CAll,settings,
              title = f'min_dist = {min_dist}, n_neighbors = {n_neighbors}',
              Figname = name+'1_UMAP_(core)')
    


# umap_params(k,MRK,dir_plots,
#             valA = min_dist, valB = n_neighbors,
#             )

# umap_params(k,MRK,dir_plots,
#             valA = 0.2, valB = 300,
#             )
# umap_params(k,MRK,dir_plots,
#             valA = 0.1, valB = 300,
#             )

# umap_params(k,MRK,dir_plots,
#             valA = 0.001, valB = 300,
#             )

In [ ]:
drawUMAP_intVals(umapData, 'sample_num',CAll,settings,
        title ='Tumor '+j+' Epigen +Cell identity UMAP', 
        Figname = name+'1_UMAP_(samples)')

### Dbscan and clusters

In [ ]:
s['min_samples']['245']=50
s['eps']['245']=0.19
s['min_samples']['1245']=70
s['eps']['1245']=0.17
min_samples = s['min_samples'][j]
eps = s['eps'][j]


dbscanData,labels,core_samples_mask = calculate_dbscan(umapData,eps=eps,min_samples=min_samples)
if visualize:
  drawDbscan(dbscanData,labels,core_samples_mask,settings,
              title=f'min_samples = {min_samples}, eps = {eps}',
              figname = name+'1_dbscan')
              
        
        

# dbscan_params(X_2d,dir_plots,
#                   valA = eps, valB = min_samples,
#                   itersA = 3, itersB = 3,
#                   )


  
 

In [ ]:
plotClusters(k,umapData,labels,NamesAll,settings,
             title = 'Clusters Tumor '+j+' - Epigen Based',
             figname = name+'1_Clusters')


## HeatMap

In [ ]:
if j=='2':
            K = k.copy() 
            k = pd.concat([k.copy(),kCD45pos2])
if visualize:
    for group_name,group in zip(groups[0],groups[1]):
    
        HeatMap(k[k.Clust!=-1],group,settings,
                title = 'T'+j+' Epigen Based: '+group_name,
                figname = name+'1_HeatMap_'+group_name)
if j=='2':
    k=K
    # del K
    K = None

## Meandist

In [ ]:
if visualize:
    for group_name,group in zip(groups[0],groups[1]):
        MeanDist(k [k.Clust==0],k [k.Clust==1],group,settings,
                title='MeanDist Epigen based: '+group_name,
                figname = name+'1_MeanDist_'+group_name)


## features on UMAP

In [ ]:

if visualize:
    drawUMAP(umapData, NamesAll,CAll,settings,
            title ='Tumor '+j+' Epigen UMAP', 
            Figname = name+'2_UMAP_')
    

## save data to file

In [ ]:
dbscanData, umapData,k, CAll,labels,s = None,None,None,None,None,None

# ALL (Epigenetics + Cell identity Based)

In [ ]:

k = k_orig.copy()
CAll=pd.concat([k]).copy()

s ={}
s['min_dist'] = {}
s['n_neighbors'] = {}
s['min_samples'] = {}
s['eps'] = {}

name = 'Tumor'+j+'_CellIden+EpiCols_'



### UMAP

plot for each feature according to its intensity

In [ ]:
s['min_dist']['245']=0.001
s['n_neighbors']['245']=10
s['min_dist']['1245']=0.001
s['n_neighbors']['1245']=10

min_dist = s['min_dist'][j]
n_neighbors = s['n_neighbors'][j]


umapData = calculate_umap(CAll[CellIden_EpiCols],n_neighbors, min_dist)
if visualize:
    drawUMAP(umapData, ['H4'],CAll,settings,
              title = f'min_dist = {min_dist}, n_neighbors = {n_neighbors}',
              Figname = name+'1_UMAP_(core)')
    


# umap_params(k,MRK,dir_plots,
#             valA = min_dist, valB = n_neighbors,
#             )

# umap_params(k,MRK,dir_plots,
#             valA = 0.2, valB = 300,
#             )
# umap_params(k,MRK,dir_plots,
#             valA = 0.1, valB = 300,
#             )

# umap_params(k,MRK,dir_plots,
#             valA = 0.001, valB = 300,
#             )

In [ ]:
drawUMAP_intVals(umapData, 'sample_num',CAll,settings,
        title ='Tumor '+j+' Epigen +Cell identity UMAP', 
        Figname = name+'1_UMAP_(samples)')

### Dbscan and clusters

In [ ]:
s['min_samples']['245']=50
s['eps']['245']=0.19
s['min_samples']['1245']=70
s['eps']['1245']=0.17


min_samples = s['min_samples'][j]
eps = s['eps'][j]


dbscanData,labels,core_samples_mask = calculate_dbscan(umapData,eps=eps,min_samples=min_samples)
if visualize:
  drawDbscan(dbscanData,labels,core_samples_mask,settings,
              title=f'min_samples = {min_samples}, eps = {eps}',
              figname = name+'1_dbscan')
              
        
        

# dbscan_params(X_2d,dir_plots,
#                   valA = eps, valB = min_samples,
#                   itersA = 3, itersB = 3,
#                   )


  
 

In [ ]:
plotClusters(k,umapData,labels,NamesAll,settings,
             title = 'Clusters Tumor '+j+' - Epigen +Cell identity Based',
             figname = name+'1_Clusters')


## HeatMap

In [ ]:
if j=='2':
            K = k.copy() 
            k = pd.concat([k.copy(),kCD45pos2])
if visualize:
    for group_name,group in zip(groups[0],groups[1]):
    
        HeatMap(k[k.Clust!=-1],group,settings,
                title = 'T'+j+' Epigen+Cell Based: '+group_name,
                figname = name+'1_HeatMap_'+group_name)  
if j=='2':
    k=K
    # del K
    K = None

## Meandist

In [ ]:

if visualize:
    for group_name,group in zip(groups[0],groups[1]):
        MeanDist(k [k.Clust==0],k [k.Clust==1],group,settings,
                title='MeanDist Epigen+Cell based: '+group_name,
                figname = name+'1_MeanDist_'+group_name)


## features on UMAP

In [ ]:

if visualize:
    drawUMAP(umapData, NamesAll,CAll,settings,
            title ='Tumor '+j+' Epigen +Cell identity UMAP', 
            Figname = name+'2_UMAP_')
    

## save data to file

In [ ]:
dbscanData, umapData,k, CAll,labels,s = None,None,None,None,None,None

# copy to single pdf

In [ ]:
dict ={}
dict['k'] = k_orig

pickle_dump('k'+j+'_sample', dict,dir_data)
imList2pdf(dir_plots,j)

end = time.process_time()
print ('total run time =', end-start )